In [17]:
import numpy as np

import numpy as np

def generate_fruit_data(n_samples):
    np.random.seed(42)
    
    # Яблоки - более четкое разделение характеристик
    apples = np.array([
        [np.random.randint(70, 80),     # размер
         np.random.randint(150, 200),   # вес
         np.random.randint(1, 3),       # цвет (1-красный, 2-зеленый)
         np.random.randint(7, 9),       # твердость
         np.random.randint(12, 14),     # содержание сахара
         np.random.randint(35, 40),     # кислотность (pH * 10)
         np.random.randint(3, 5)]       # шероховатость
        for _ in range(n_samples)])
    
    # Апельсины
    oranges = np.array([
        [np.random.randint(65, 70),     # размер
         np.random.randint(140, 150),   # вес
         3,                             # цвет (3-оранжевый)
         np.random.randint(4, 6),       # твердость
         np.random.randint(8, 12),      # содержание сахара
         np.random.randint(30, 35),     # кислотность (pH * 10)
         np.random.randint(8, 9)]       # шероховатость
        for _ in range(n_samples)])
    
    # Груши
    pears = np.array([
        [np.random.randint(80, 90),     # размер
         np.random.randint(190, 250),   # вес
         2,                             # цвет (2-зеленый)
         np.random.randint(5, 7),       # твердость
         np.random.randint(14, 16),     # содержание сахара
         np.random.randint(38, 42),     # кислотность (pH * 10)
         np.random.randint(6, 7)]       # шероховатость
        for _ in range(n_samples)])
    
    X = np.vstack((apples, oranges, pears))
    y = np.array([0] * n_samples + [1] * n_samples + [2] * n_samples)
    
    X = ((X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0)) * 255).astype(np.uint16)
    return X, y

def relu(x):
    return np.maximum(0, x)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x // np.sum(exp_x, axis=1, keepdims=True)

class SimpleNeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.W1 = np.random.randint(-2147483648, 2147483647, (input_size, hidden_size), dtype=np.int32)
        self.W2 = np.random.randint(-2147483648, 2147483647, (hidden_size, output_size), dtype=np.int32)
        
    def forward(self, X):
        self.hidden = relu(np.dot(X, self.W1))
        self.output = np.dot(self.hidden, self.W2)
        return self.output
    
    def train(self, X, y, learning_rate=1, epochs=100):
        for epoch in range(epochs):
            hidden = relu(np.dot(X, self.W1))
            output = np.dot(hidden, self.W2)
            
            output_error = output.copy()
            output_error[range(len(y)), y] -= 1
            
            W2_update = np.dot(hidden.T, output_error)
            W1_update = np.dot(X.T, np.dot(output_error, self.W2.T) * (hidden > 0))
            
            self.W2 -= np.round(learning_rate * W2_update).astype(dtype=np.int32)
            self.W1 -= np.round(learning_rate * W1_update).astype(dtype=np.int32)
    
    def predict(self, X):
        output = self.forward(X)
        return np.argmax(output, axis=1)


X, y = generate_fruit_data(1000)


indices = np.random.permutation(len(X))
train_size = int(0.8 * len(X))

train_indices, test_indices = indices[:train_size], indices[train_size:]

X_train, y_train = X[train_indices], y[train_indices]
X_test, y_test = X[test_indices], y[test_indices]

nn = SimpleNeuralNetwork(input_size=7, hidden_size=12, output_size=3)

nn.train(X_train, y_train, learning_rate=2000, epochs=100)
predictions = nn.predict(X_test)
accuracy = np.mean(predictions == y_test)

# Вывод весов для последующей реализации в Verilog
print("\nВеса первого слоя:")
print(nn.W1)
print("\nВеса второго слоя:")
print(nn.W2)
print(f"Точность на тестовой выборке: {accuracy * 100:.2f}%")


Веса первого слоя:
[[ 1854000351  -196648100 -1125545277 -1705988205 -1601524694    16162507
   1951660932  -199914819  1257503993  1626299518   809645384    60280764]
 [-1646527913 -1798709546  1102161284 -1289108936  1105286360   996257817
    122358201  1053608897   165821587 -2108426057  1434297295 -1340806916]
 [-1607856195 -1720339360  2072750172 -1692089890   401252228 -1478290649
   -522241199   677162289  -217103748  2002122287  -893590801  -767593001]
 [ 1259805513 -1394345057  1940597115  1366368152   443106566   447016855
    193255268  -497413184 -1824980205  1703617534  1182908065   309223906]
 [-1179681013  1771327090 -1354891768 -1322590900 -1609370594 -1184645303
   1488557534  1374524403  1498386033  1903853858  1345704178 -2128823331]
 [ 2062223574  1543841207  1802325884   616476159  2018913914   829738684
   -852522267  1984899731   983882437 -1541850484 -1722913654 -1229932448]
 [-1134801652  -965664455 -1112723209 -1735325872 -1038690802  1785482740
    86550947